In [ ]:
pip install -U langchain langchain-community langchain-core langchain-classic langchain_chroma langchain-huggingface langchain-google-genai langchain-groq

In [ ]:
import os
from langchain_classic.chains import create_history_aware_retriever, create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq

In [157]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [158]:
vector_db = Chroma(
    persist_directory="my_vectordb", 
    embedding_function=embeddings,
    collection_name="office_docs"
)

In [ ]:
os.environ["GROQ_API_KEY"] = "key"

In [161]:
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0)

In [162]:
retriever = vector_db.as_retriever(search_kwargs={"k": 3})

In [163]:
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood, "
    "If user can answer the user Reminder points"
)

In [164]:
contextualize_q_prompt = ChatPromptTemplate.from_messages([
    ("system", contextualize_q_system_prompt),
    MessagesPlaceholder("chat_history"), # Dynamic history goes here
    ("human", "{input}"),
])

In [165]:
history_aware_retriever = create_history_aware_retriever(llm, retriever, contextualize_q_prompt)

In [166]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer the question. "
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

In [167]:
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

In [168]:
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [169]:
store = {}

In [170]:
def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [171]:
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [189]:
user_1_config = {"configurable": {"session_id": "user_123"}}
user_2_config = {"configurable": {"session_id": "user_124"}}

In [ ]:
res1 = conversational_rag_chain.invoke({"input": "What is the office location?"}, config=user_1_config)
print(f"User 1: What is the office location?\nAI: {res1['answer']}\n")

User: What is the office location?
AI: Our office is located in Bangalore, India.



In [196]:
res2 = conversational_rag_chain.invoke({"input": "What country is that city in?"}, config=user_1_config)
print(f"User 1: What country is that city in?\nAI: {res2['answer']}")

User 1: What country is that city in?
AI: As I mentioned earlier, Bangalore is in India.


In [195]:
res3 = conversational_rag_chain.invoke({"input": "Remember this Number - 3"}, config=user_1_config)
print(f"User 1: Remember this Number\nAI: {res3['answer']}")

User 1: Remember this Number
AI: Still remembered: 3.


In [194]:
res4 = conversational_rag_chain.invoke({"input": "What number I asked you to remember?"}, config=user_1_config)
print(f"User 1: What number I asked you to remember?\nAI: {res4['answer']}")

User 1: What number I asked you to remember?
AI: The number you asked me to remember is 3.


In [193]:
res5 = conversational_rag_chain.invoke({"input": "What number I asked you to remember?"}, config=user_2_config)
print(f"User 2: What number I asked you to remember?\nAI: {res5['answer']}")

User 2: What number I asked you to remember?
AI: You still haven't asked me to remember any number. Our conversation only mentions the office location and the CEO's name.


In [188]:
store

{'user_123': InMemoryChatMessageHistory(messages=[HumanMessage(content='What is the office location?', additional_kwargs={}, response_metadata={}), AIMessage(content='Our office is located in Bangalore.', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]), HumanMessage(content='What country is that city in?', additional_kwargs={}, response_metadata={}), AIMessage(content='Bangalore is a city in India.', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]), HumanMessage(content='What is the office location?', additional_kwargs={}, response_metadata={}), AIMessage(content='Our office is located in Bangalore, India.', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]), HumanMessage(content='What country is that city in?', additional_kwargs={}, response_metadata={}), AIMessage(content='As I mentioned earlier, Bangalore is in India.', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_